In [37]:
import json
from slugify import slugify

with open("gliderbase.json") as file:
    parsed_json = json.load(file)

In [10]:
glider_list = parsed_json["result"]["data"]["allGlidersJson"]["edges"]
manufacturers = dict([(x["node"]["manufacturer"]["slug"], x["node"]["manufacturer"]) for x in glider_list])
manufacturers

{'advance': {'name': 'Advance', 'slug': 'advance', 'altNames': None},
 'aircross': {'name': 'Aircross',
  'slug': 'aircross',
  'altNames': ['Air Cross']},
 'airdesign': {'name': 'AirDesign',
  'slug': 'airdesign',
  'altNames': ['AD', 'Air Design']},
 'axis': {'name': 'Axis',
  'slug': 'axis',
  'altNames': ['Axis Para', 'axispara']},
 'bgd': {'name': 'Bruce Goldsmith Design', 'slug': 'bgd', 'altNames': ['BGD']},
 'dudek': {'name': 'Dudek', 'slug': 'dudek', 'altNames': None},
 'flow': {'name': 'Flow', 'slug': 'flow', 'altNames': ['Flow Paragliders']},
 'gin': {'name': 'Gin', 'slug': 'gin', 'altNames': ['Gin Gliders']},
 'icaro': {'name': 'Icaro',
  'slug': 'icaro',
  'altNames': ['Icaro Paragliders']},
 'mac-para': {'name': 'Mac Para', 'slug': 'mac-para', 'altNames': ['MacPara']},
 'niviuk': {'name': 'Niviuk', 'slug': 'niviuk', 'altNames': None},
 'nova': {'name': 'Nova', 'slug': 'nova', 'altNames': None},
 'ozone': {'name': 'Ozone', 'slug': 'ozone', 'altNames': None},
 'papillon-para

In [46]:
def extract_glider(entry):
    data = entry["node"]
    #     string_columns = ["manufacturer", "name", "size", "certification", "display_name", "cell_count"]
    #     range_columns = ["certified_weight_range", "recommended_weight_range", "extended_weight_range"]
    #     numeric_columns = ["projected_span", "projected_area", "projected_aspect_ratio", "flat_span", "flat_area", "flat_aspect_ratio", "chord", "weight"]
    generic_data = {
        "manufacturer": data["manufacturer"]["slug"],
        "name": data["slug"],
        "display_name": data["name"],
    }
    
    for size in data["sizes"]:
        attributes = dict([(x["name"] + str(x["additionalInfo"] or ""), x) for x in size["attributes"]])
        glider = generic_data.copy()
        glider["size"] = size["name"]
        if "Certification" in attributes:
            glider["certification"] = attributes["Certification"]["valueString"]
        if "Cells" in attributes:
            glider["cell_count"] = attributes["Cells"]["valueString"] or f"{attributes['Cells']['valueNumber']}"
        if "Projected span" in attributes:
            glider["projected_span"] = attributes["Projected span"]["valueNumber"]
        if "Projected area" in attributes:
            glider["projected_area"] = attributes["Projected area"]["valueNumber"]
        if "Projected aspect ratio" in attributes:
            glider["projected_aspect_ratio"] = attributes["Projected aspect ratio"]["valueNumber"]
        if "Flat span" in attributes:
            glider["flat_span"] = attributes["Flat span"]["valueNumber"]
        if "Flat area" in attributes:
            glider["flat_area"] = attributes["Flat area"]["valueNumber"]
        if "Flat aspect ratio" in attributes:
            glider["flat_aspect_ratio"] = attributes["Flat aspect ratio"]["valueNumber"]
        if "Chord" in attributes:
            glider["chord"] = attributes["Chord"]["valueNumber"]
        if "Glider weight" in attributes:
            glider["weight"] = attributes["Glider weight"]["valueNumber"]
        if "Takeoff weight" in attributes and attributes["Takeoff weight"]["valueRange"]:
            glider["certified_weight_range"] = (attributes["Takeoff weight"]["valueRange"]["from"], attributes["Takeoff weight"]["valueRange"]["to"])
        if "Takeoff weightrecommended" in attributes and attributes["Takeoff weightrecommended"]["valueRange"]:
            glider["recommended_weight_range"] = (attributes["Takeoff weightrecommended"]["valueRange"]["from"], attributes["Takeoff weightrecommended"]["valueRange"]["to"])
        if "Takeoff weightextended" in attributes and attributes["Takeoff weightextended"]["valueRange"]:
            glider["extended_weight_range"] = (attributes["Takeoff weightextended"]["valueRange"]["from"], attributes["Takeoff weightextended"]["valueRange"]["to"])
        
        yield glider

In [48]:
gliders = [extract_glider(data) for data in glider_list]
flat_gliders = [item for sublist in gliders for item in sublist]
len(flat_gliders)

1320